## RNN from scratch

### Language Model

* At each time-step, the RNN tries to predict what the next character is, given the previous characters. 
* $\mathbf{X} = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ is a list of characters from the training set.
* $\mathbf{Y} = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$ is the same list of characters but shifted one character forward. 

<img src="images/rnn.png" style="width:450;height:300px;">
<caption><center><font color='purple'><b>Figure 1</b>: Recurrent Neural Network </center></caption></img>

### Sampling a New Sequence

<img src="images/dinos3.png" style="width:500;height:300px;">
<caption><center><font color='purple'><b>Figure 3</b>:Pass in $x^{\langle 1\rangle} = \vec{0}$ at the first time-step, and have the network sample one character at a time. </center></caption></img>

*hidden state:*  
$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t+1 \rangle } + W_{aa} a^{\langle t \rangle } + b)\tag{1}$$
*activation:*
$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2}$$
*prediction:*
$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle })\tag{3}$$
where $x^{\langle 1 \rangle} = \vec{0}$ and $a^{\langle 0 \rangle} = \vec{0}$

## Helper Functions
---

In [1]:
import numpy as np
import random
import copy

In [2]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

### Sampling a Sequence

In [3]:
def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters from a trained RNN
    """
    
    # Retrieve parameters and shapes
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # Initialize 'x' and 'a' vectors
    x = np.zeros((vocab_size, 1))
    a_prev = np.zeros((n_a, 1))
    
    # List to hold the output
    indices = []   
    
    # Stopping condition - length of 50 or newline character
    idx = -1       # Initialize idx to non-zero (zero holds newline character)
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Forward propagate
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba)
        z = np.dot(Wya, a) + by
        y = softmax(z)

        # Pick a character
        idx = np.random.choice(range(len(y.ravel())), p=y.ravel())
        indices.append(idx)
        
        # Update values for next step
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        a_prev = a
        
        counter +=1

    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    name = ''.join([ix_to_char[i] for i in indices])
    name = name[0].upper() + name[1:]
    
    return indices, name

### Clipping Gradients

* Exploding gradients make training difficult, because the updates may be so large that they "overshoot" the optimal values during back propagation.
* Before updating the parameters, we will perform gradient clipping


<img src="images/clip.png" style="width:400;height:150px;"></img>

In [4]:
def clip(gradients, val):
    '''
    Clips the gradients' values b/w provided limit.
    '''
    gradients = copy.deepcopy(gradients)
    
    dWaa, dWax, dWya, dba, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['dba'], gradients['dby']
   
    for gradient in [dWaa, dWax, dWya, dba, dby]:
        np.clip(gradient, -val, val, out = gradient)

    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "dba": dba, "dby": dby}
    return gradients

### Forward & Backward Propagation

**Dimensions**:
* Wax -- Weight matrix for the input (xt), numpy array of shape (n_a, n_x)
* Waa -- Weight matrix for the hidden state (at), numpy array of shape (n_a, n_a)
* Wya -- Weight matrix for activations to the output, numpy array of shape (n_y, n_a)
* ba -- Bias for hidden state, numpy array of shape (n_a, 1)
* by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
* xt of shape (n_x, m).
* at of shape (n_a, m)
* yt of shape (n_y, m)

#### **NOTE 1**:
* We don't cast X into (n_x, m, T_x) tensor, and train over each sample individually, where each sample has a diff. length.
* Instead of converting X into One-Hot vector matrix at the beginning, we handle it in Forward Pass step

#### **NOTE 2**:
* Gradients are updated for each alphabet in the word (or each word in the sentence)
* However, Parameters are updates only 1x per word (in this case) or (more generally) 1x/batch

In [5]:
def initialize_parameters(n_a, n_x, n_y):
    """
    Initialize parameters with small random values
    """
    np.random.seed(1)
    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    ba = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba,"by": by}
    return parameters

In [6]:
def rnn_step_forward(parameters, a_prev, xt):
    """
    Implements a single forward step of the RNN-cell
    """
    Wax, Waa, Wya = parameters["Wax"], parameters["Waa"], parameters["Wya"]
    ba, by = parameters["ba"], parameters["by"]
    
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    p_t = softmax(np.dot(Wya, a_next) + by)

    #cache = (a_next, a_prev, xt, parameters)    # dict is built in rnn_forward function instead
    return a_next, p_t

In [7]:
def rnn_forward(X, Y, a0, parameters, n_x = 27):
    """
    Implements one full forward pass
    """
    # Initialize x, a and y_hat as dictionaries to hold values for each step
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    loss = 0             # initialize your loss to 0
    
    for t in range(len(X)):
        # ONE-HOT VECTOR TRANSFORMATION
        val = X[t]
        x[t] = np.zeros((n_x,1)) 
        if (val != None):
            x[t][val] = 1
        
        # Run one step forward of the RNN
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        
        # Update the loss by substracting the cross-entropy term of this time-step from it.
        loss -= np.log(y_hat[t][Y[t],0])   # y*log(y_hat) = log( y_hat[relevant_index] )
        
    cache = (y_hat, a, x)     
    return loss, cache

In [8]:
def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['dba'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients

In [9]:
def rnn_backward(X, Y, parameters, cache):
    
    # Retrieve from cache and parameters
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    
    # Initialize gradients as an empty dictionary
    gradients = {}
    
    # Initializing gradients
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['dba'], gradients['dby'] = np.zeros_like(ba), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    
    # Backpropagate through time
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a

In [10]:
def update_parameters(parameters, gradients, lr):
    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['ba']  += -lr * gradients['dba']
    parameters['by']  += -lr * gradients['dby']
    return parameters

In [11]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    X -- one example word representated as a list of integers
    Y -- exactly the same as X but shifted one index to the left.
    
    loss -- value of the loss function (cross-entropy)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    loss, cache  = rnn_forward(X, Y, a_prev, parameters)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    gradients    = clip(gradients, 5)
    parameters   = update_parameters(parameters, gradients, learning_rate)
    
    return loss, gradients, a[len(X)-1]

---
## Dinosaur Names

In [12]:
data = open('data/rnn/dinos.txt', 'r').read()
data = data.lower()

In [13]:
chars = sorted(list(set(data)))
vocab_size = len(chars)
print('Vocab size: ', vocab_size)
print('Characters: ', chars)

Vocab size:  27
Characters:  ['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [14]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [15]:
data = data.split('\n')
data = [line.strip() for line in data]
data[:5]

['aachenosaurus', 'aardonyx', 'abdallahsaurus', 'abelisaurus', 'abrictosaurus']

In [16]:
data_ch = [list(name) for name in data]
data_ch[:2]

[['a', 'a', 'c', 'h', 'e', 'n', 'o', 's', 'a', 'u', 'r', 'u', 's'],
 ['a', 'a', 'r', 'd', 'o', 'n', 'y', 'x']]

In [17]:
np.random.seed(0)
np.random.shuffle(data_ch)

In [18]:
data_int = []
for i in range(len(data_ch)):
    data_int.append([char_to_ix[c] for c in data_ch[i]])
    
print(data_ch[:1], '\n', data_int[:2])

[['t', 'u', 'r', 'i', 'a', 's', 'a', 'u', 'r', 'u', 's']] 
 [[20, 21, 18, 9, 1, 19, 1, 21, 18, 21, 19], [16, 1, 14, 4, 15, 18, 1, 22, 5, 14, 1, 20, 15, 18]]


---
## Generating New Names (after Training Model)

In [19]:
def model(data_x, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27, verbose = False):
    """
    Trains the model and generates dinosaur names. 
    
    Arguments:
    data_x -- text corpus (list of integers)
    ix_to_char -- maps the index to a character
    char_to_ix -- maps a character to an index
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names to sample at each iteration. 
    
    Returns:
    parameters -- learned parameters
    """
    
    # Initialize shapes, parameters
    n_x, n_y = vocab_size, vocab_size
    m = len(data_x)
    parameters = initialize_parameters(n_a, n_x, n_y)
    a_prev = np.zeros((n_a, 1))
    
    # Initialize loss (this is required because we want to smooth our loss)
    loss = -np.log(1.0/vocab_size)*dino_names
    
    # Optimization loop
    for j in range(num_iterations):
        
        # Select an example and set X & Y
        idx = j % m
        X = [None] + data_x[idx]
        Y = data_x[idx] + [char_to_ix['\n']]

        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)
        
        # to keep the loss smooth.
        loss = loss*0.999 + curr_loss*0.001

        # Check every 2000 Iteration
        if (j > 0) and (j % 5000 == 0):  
            print('Iteration: {}, Loss: {:.4f}'.format(j, loss) + '\n')
            
            seed = 0
            for name in range(dino_names):
                sampled_ix, sampled_name = sample(parameters, char_to_ix, seed)
                print(sampled_name.replace('\n',''))  # removes extra line
                seed += 1
            print('\n')
        
    return parameters

In [20]:
params = model(data_int, ix_to_char, char_to_ix, 35001, verbose = True, dino_names=3)

Iteration: 5000, Loss: 25.2018

Encaradudon
Eosaurus
Noopeodhenwanrmolongastia


Iteration: 10000, Loss: 23.8439

Annabverathaurorhatophnimidraithomys
Lolomalesmphocenioodenitan
Palsiia


Iteration: 15000, Loss: 23.0549

Erathosaurus
Banopalren
Pholonobor


Iteration: 20000, Loss: 23.0673

Teohushdewlopestyluguantosaurus
Stoceshaegstroclolisaurus
Mopachanglosaurus


Iteration: 25000, Loss: 22.7241

Agngnrang
Oluasaurus
Mantasia


Iteration: 30000, Loss: 22.7230

Inchusaurus
Licampalisaurus
S


Iteration: 35000, Loss: 22.3328

Henreosaurus
Ychosaurus
Chadiys




---
## Shakespeare Sonet Example

A similar task to character-level text generation (but more complicated) is generating Shakespearean poems. Instead of learning from a dataset of dinosaur names, we can use a collection of Shakespearean poems. 

Using LSTM cells, we can learn longer-term dependencies that span many characters in the text--e.g., where a character appearing somewhere a sequence can influence what should be a different character, much later in the sequence.

<img src="images/shakespeare.jpg" style="width:500;height:400px;"></img>

In [21]:
from tensorflow.keras.models import load_model
import numpy as np
import random
import sys
import io

In [22]:
text = io.open('data/rnn/shakespeare.txt', encoding='utf-8').read().lower()
print('corpus length:', len(text))
print(text[100:180])

corpus length: 94275
s rose might never die,
but as the riper should by time decease,
his tender heir


In [23]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
print('number of unique characters in the corpus:', len(chars))   # becomes n_x

n_x = len(chars)

number of unique characters in the corpus: 38


In [24]:
def build_data(text, Tx = 40, stride = 3):
    """
    Create a training set by scanning a window of size Tx over the text corpus, with stride 3.
    """
    X, Y = [], []
    for i in range(0, len(text) - Tx, stride):
        X.append(text[i: i + Tx])
        Y.append(text[i + Tx])    # next alphabet in the sequence
    
    print('number of training examples:', len(X))
    return X, Y

In [25]:
def vectorization(X, Y, n_x, char_indices, Tx = 40):
    """
    Convert X and Y (lists) into arrays to be given to a recurrent neural network.
    
    Returns:
    x -- array of shape (m, Tx, n_x)
    y -- array of shape (m, n_x)
    """
    
    m = len(X)
    x = np.zeros((m, Tx, n_x), dtype=np.bool)
    y = np.zeros((m, n_x), dtype=np.bool)
    for i, sentence in enumerate(X):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[Y[i]]] = 1
    return x, y 

In [26]:
Tx = 40
stride  = 3

X, Y = build_data(text, Tx, stride)
print(len(X), len(Y))

number of training examples: 31412
31412 31412


In [27]:
X[:2], Y[:2]

(['the sonnets\n\nby william shakespeare\n\nfro',
  ' sonnets\n\nby william shakespeare\n\nfrom f'],
 ['m', 'a'])

In [28]:
x, y = vectorization(X, Y, n_x, char_indices = char_indices) 
print(x.shape, y.shape)

(31412, 40, 38) (31412, 38)


In [29]:
model = load_model('pretrainedmodel/rnn_shakespeare/model_shakespeare_kiank_350_epoch.h5')
model.fit(x, y, batch_size=128, epochs=1)

246/246 [==============================] - 35s 141ms/step - loss: 2.5554


In [30]:
def sample1(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(n_x), p = probas.ravel())
    return out

In [31]:
def generate_poem():
    usr_input = input("Write the beginning for a poem, and let Shakespeare complete it: ")
    
    # zero pad the sentence to Tx characters.
    sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    print('Padded sentence: ', sentence)
    
    generated = usr_input 

    print("\n\nHere is your poem (next 400 characters): \n\n") 
    print(usr_input)
    
    for i in range(400):
        x_pred = np.zeros((1, Tx, n_x))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1.
        
        # Generate prediction for next character
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample1(preds, temperature = 1.0)
        next_char = indices_char[next_index]
        
        # Add that to the sentence
        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

In [32]:
generate_poem()

Write the beginning for a poem, and let Shakespeare complete it: Thou lips are like a summer breeze,
Padded sentence:  00000thou lips are like a summer breeze,


Here is your poem (next 400 characters): 


Thou lips are like a summer breeze,
 toof shen formited,
shanth pout deder's wiet-trom bnhound my eveny from love,
thy fore misein  in that comcost to,,
that hit try frormar wastes of my keans being,
youren to the mray me your cruth in they see,
shale the mvay of this,  all dine newil,
not sanse and i strowmed wuld, cortet i sind
and dos the prairor wind,
is goeteds a sach fros as forsed,
thus and plerannery tompary nehed,
who nef m